## Setup

### Packages

- `awkward`: For dealing with nested, variable-sized data.
- `pennylane`: Quantum machine learning.
- `lightning`: Simplifying training process.
- `pytorch_geometric`: Graph neural network package.
- `wandb`: Monitoring training process.

In [1]:
# basic packages
import os, time, random
from itertools import product
import matplotlib.pyplot as plt

# data
import awkward as ak
from d_hep_data import JetEvents

# qml
import pennylane as qml
from pennylane import numpy as np

# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

# pytorch_lightning
import lightning as L
import lightning.pytorch as pl
from lightning.pytorch.tuner import Tuner

# pytorch_geometric
import networkx as nx
import torch_geometric.nn as geom_nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing

# wandb
import wandb
from lightning.pytorch.loggers import WandbLogger
wandb.login()

# reproducibility
L.seed_everything(3020616)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# faster calculation on GPU but less precision
torch.set_float32_matmul_precision("medium")

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------


/home/yianchen/.pyenv/versions/3.9.12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ntuyianchen. Use `wandb login --relogin` to force relogin
INFO: Global seed set to 3020616
INFO:lightning.fabric.utilities.seed:Global seed set to 3020616


### Configurations

Hyperparameters and configurations for:
- Data (channel, .etc)
- Training process (Trainer, .etc)
- Model architecture (input/output dimension, .etc)

In [2]:
# configuration dictionary
cf = {}
cf["time"]    = time.strftime("%Y%m%d_%H%M%S", time.localtime())
cf["wandb"]   = True
cf["project"] = "t_4vec_2pcnn"

# data infotmation
cf["num_events"]    = "5000"
cf["sig_channel"]   = "ZprimeToZhToZinvhbb"
cf["bkg_channel"]   = "QCD_HT2000toInf"
cf["jet_type"]      = "fatjet"
cf["cut"]           = f"({cf['jet_type']}_pt>=500)&({cf['jet_type']}_pt<=1500)"
cf["subjet_radius"] = 0.25

# traning configuration
cf["num_train_ratio"]   = 0.5
cf["num_test_ratio"]    = 0.5
cf["batch_size"]        = 16
cf["num_workers"]       = 12
cf["max_epochs"]        = 50
cf["accelerator"]       = "gpu" if torch.cuda.is_available() else "cpu"
cf["num_data"]          = 1000
cf["fast_dev_run"]      = False
cf["log_every_n_steps"] = cf["batch_size"] // 2

# model hyperparameters
cf["loss_function"]  = nn.BCEWithLogitsLoss()
cf["optimizer"]      = optim.Adam
cf["learning_rate"]  = 1E-3

# 2PCNN hyperparameters
cf["gnn_layers"] = 1
cf["mlp_layers"] = 2

### Data Module

In this project, we train with data containing only the four momentum of particles. In order to reduce the size of the data (due to the long training time for quantum machine learning), we reduce the size of data by `fastjet` package by clustering particles again by `anti-kt algorithm` with smaller radius.

The detail (source code) for creating fastjet reclustering events is in the `d_hep_data` file.

To test the power of QML for learning space structure of data (geometric angles, e.g. $p_t$, $\eta$, $\phi$), we will use four momentum only (or z-boosted invariant variables $p_t$, $\eta$, $\phi$).

In [3]:
def get_events_dict(channel):
    jet_events     = JetEvents(cf[f"{channel}_channel"], cf["num_events"], cf["jet_type"], cf["cut"])
    fastjet_events = jet_events.fastjet_events(R=cf["subjet_radius"])
    events_dict = {
        # parent jet features
        "e":jet_events.events[f"{cf['jet_type']}_e"],
        "pt":jet_events.events[f"{cf['jet_type']}_pt"],
        "eta":jet_events.events[f"{cf['jet_type']}_eta"],
        "phi":jet_events.events[f"{cf['jet_type']}_phi"],
        "px":jet_events.events[f"{cf['jet_type']}_px"],
        "py":jet_events.events[f"{cf['jet_type']}_py"],
        "pz":jet_events.events[f"{cf['jet_type']}_pz"],
        # fastjet daughter features
        "_e":fastjet_events["e"],
        "_pt":fastjet_events["pt"],
        "_eta":fastjet_events["eta"],
        "_phi":fastjet_events["phi"],
        "_theta":fastjet_events["theta"],
        "_px":fastjet_events["px"],
        "_py":fastjet_events["py"],
        "_pz":fastjet_events["pz"],
        "_delta_eta":fastjet_events["delta_eta"],
        "_delta_phi":fastjet_events["delta_phi"],
    }
    events_dict["p"]     = (events_dict["px"]**2 + events_dict["py"]**2 + events_dict["pz"]**2) ** 0.5
    events_dict["theta"] = np.arccos(events_dict["pz"]/events_dict["p"])
    return events_dict

sig_events_dict = get_events_dict("sig")
bkg_events_dict = get_events_dict("bkg")

DataLog: Successfully create ZprimeToZhToZinvhbb with 2980 events.
DataLog: Finish reclustering ZprimeToZhToZinvhbb with anti-kt algorithm.
DataLog: Successfully create QCD_HT2000toInf with 4917 events.
DataLog: Finish reclustering QCD_HT2000toInf with anti-kt algorithm.


In [4]:
def ak_to_torch(arrays):
    x = []
    num_events = len(arrays[0])
    for i in range(num_events):
        tensors = [torch.tensor(a[i], dtype=torch.float32).reshape(-1, 1) for a in arrays]
        x.append(torch.cat(tensors, axis=1))
    return x

def events_pt_eta_phi(events_dict, norm):
    if norm:
        arrays = ak.broadcast_arrays(
            4 * np.arctan(events_dict["_pt"] / events_dict["pt"]),
            4 * np.arctan(events_dict["_delta_eta"]),
            2 * events_dict["_delta_phi"],
        )
    else:
        arrays = ak.broadcast_arrays(
            events_dict["_pt"],
            events_dict["_delta_eta"],
            events_dict["_delta_phi"],
        )
    return ak_to_torch(arrays)

In [5]:
class JetDataModule(pl.LightningDataModule):
    def __init__(self, events_func, tune_batch=False, **kwargs):
        '''Add a "_" prefix if it is a fastjet feature'''
        super().__init__()
        self.batch_size = 1 if tune_batch else cf["batch_size"]

        # jet events
        sig_events = events_func(sig_events_dict, **kwargs)
        bkg_events = events_func(bkg_events_dict, **kwargs)
        self.sig_data_list = self._create_data_list(sig_events, 1)
        self.bkg_data_list = self._create_data_list(bkg_events, 0)

        # count the number of training, and testing
        assert len(self.sig_data_list) >= cf["num_data"], f"sig data not enough: {len(self.sig_data_list)} < {cf['num_data']}"
        assert len(self.bkg_data_list) >= cf["num_data"], f"bkg data not enough: {len(self.bkg_data_list)} < {cf['num_data']}"
        num_train = int(cf["num_data"] * cf["num_train_ratio"])
        num_test  = int(cf["num_data"] * cf["num_test_ratio"])
        print(f"DataLog: {cf['sig_channel']} has {len(self.sig_data_list)} events and {cf['bkg_channel']} has {len(self.bkg_data_list)} events.")
        print(f"Choose num_data for each channel to be {cf['num_data']} | Each channel  has num_train = {num_train}, num_test = {num_test}")

        # prepare dataset for dataloader
        train_idx = num_train
        test_idx  = num_train + num_test
        self.train_dataset = self.sig_data_list[:train_idx] + self.bkg_data_list[:train_idx]
        self.test_dataset  = self.sig_data_list[train_idx:test_idx] + self.bkg_data_list[train_idx:test_idx]
    
    def _create_data_list(self, events, y):
        # create pytorch_geometric "Data" object
        data_list = []
        for i in range(len(events)):
            x = events[i]
            edge_index = list(product(range(len(x)), range(len(x))))
            edge_index = torch.tensor(edge_index).transpose(0, 1)
            x.requires_grad, edge_index.requires_grad = False, False
            data_list.append(Data(x=x, edge_index=edge_index, y=y))
        random.shuffle(data_list)
        return data_list

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=cf["num_workers"],  shuffle=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=cf["num_workers"])

## Models

To compare classical GNN with quantum GNN, we use `GraphConv` and `MessagePassing` with `pennylane` for classical and quantum repectively.

- Why using `nn.ModuleList` instead of `nn.Sequential`?
Both `nn.ModuleList` and `nn.Sequential` trace the trainable parameters autometically. However, since we are using "gnn" layers, we need to feed into additional argument `edge_index`. In order to check whether we are using "gnn" layers or not, we use `isinstance` to check the class type (Since all PyTorch Geometric graph layer inherit the class `MessagePassing`). For detail, see [When should I use nn.ModuleList and when should I use nn.Sequential?](https://discuss.pytorch.org/t/when-should-i-use-nn-modulelist-and-when-should-i-use-nn-sequential/5463/3)

### MLP Layers

In [6]:
class ClassicalMLP(nn.Module):
    def __init__(self, in_channel, out_channel, hidden_channel, num_layers):
        super().__init__()
        if num_layers == 0:
            self.net = nn.Linear(in_channel, out_channel)
        else:
            net = [nn.Linear(in_channel, hidden_channel), nn.ReLU()]
            for _ in range(num_layers-2):
                net += [nn.Linear(hidden_channel, hidden_channel), nn.ReLU()]
            net += [nn.Linear(hidden_channel, out_channel)]
            self.net = nn.Sequential(*net)
    def forward(self, x):
        return self.net(x)
    
class QuantumMLP(nn.Module):
    def __init__(self, num_qubits, num_layers, num_reupload, measurements):
        super().__init__()
        # create a quantum MLP
        @qml.qnode(qml.device('default.qubit', wires=num_qubits))
        def circuit(inputs, weights):
            for i in range(num_reupload):
                qml.AngleEmbedding(features=inputs, wires=range(num_qubits), rotation='Y')
                qml.StronglyEntanglingLayers(weights=weights[i], wires=range(num_qubits))
            measurements_dict = {"X":qml.PauliX, "Y":qml.PauliY, "Z":qml.PauliZ}
            return [qml.expval(measurements_dict[m[1]](wires=m[0])) for m in measurements]
        # turn the quantum circuit into a torch layer
        weight_shapes = {"weights":(num_reupload, num_layers, num_qubits, 3)}
        net = [qml.qnn.TorchLayer(circuit, weight_shapes=weight_shapes)]
        self.net = nn.Sequential(*net)
    def forward(self, x):
        return self.net(x)

### Classical 2PCNN

In [7]:
class Classical2PCNNForwardMP(MessagePassing):
    def __init__(self, num_features, num_layers, aggr):
        super().__init__(aggr=aggr)
        self.mp_phi = ClassicalMLP(
            in_channel     = 2*num_features, 
            out_channel    = num_features,
            hidden_channel = 2*num_features,
            num_layers     = num_layers,
            )
        self.mp_gamma = ClassicalMLP(
            in_channel     = 2*num_features, 
            out_channel    = num_features,
            hidden_channel = 2*num_features,
            num_layers     = num_layers,
        )
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    def message(self, x_i, x_j):
        return self.mp_phi(torch.cat((x_i, x_j), dim=-1))
    def update(self, aggr_out, x):
        return self.mp_gamma(torch.cat((x, aggr_out), dim=-1))

class Classical2PCNNForward(nn.Module):
    def __init__(self, gnn_in, gnn_layers, gnn_aggr, mlp_in, mlp_out, mlp_hidden, mlp_layers):
        super().__init__()
        self.gnn = Classical2PCNNForwardMP(gnn_in, gnn_layers, gnn_aggr)
        self.mlp = ClassicalMLP(mlp_in, mlp_out, mlp_hidden, mlp_layers)
    def forward(self, x, edge_index, batch):
        x = self.gnn(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch)
        x = self.mlp(x)
        return x

### Lightning Module

Most of the hyperparameters are defined at `cf` configuration dictionary.

Note that when using `nn.BCEWithLogitsLoss`, the first argument should not be paased to `sigmoid`.

In [8]:
class LitModel(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.save_hyperparameters(ignore=['model'])
        self.model = model
        self.loss_function = cf["loss_function"]

    def forward(self, data):
        # predict y
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.model(x, edge_index, batch)
        x = x.squeeze(dim=-1)

        # calculate loss and accuracy
        y_pred = x > 0
        y_true = data.y
        loss   = self.loss_function(x, y_true.float())
        acc    = (y_pred == data.y).float().mean()
        return loss, acc

    def configure_optimizers(self):
        optimizer = cf["optimizer"](self.parameters(), lr=cf["learning_rate"])
        return optimizer

    def training_step(self, data, batch_idx):
        loss, acc = self.forward(data)
        self.log("train_loss", loss, on_step=True, on_epoch=True, batch_size=len(data.x))
        self.log("train_acc", acc, on_step=True, on_epoch=True, batch_size=len(data.x))
        return loss

    def test_step(self, data, batch_idx):
        _, acc = self.forward(data)
        self.log("test_acc", acc, on_step=True, on_epoch=True, batch_size=len(data.x))

## Train/Test the Model

### Training procedure

In [9]:
def train(model, data_module, tune_batch=False, suffix=""):
    # setup id and path for saving
    project  = cf['project']
    group    = f"{cf['num_data']}_{cf['sig_channel']}_{cf['bkg_channel']}_{cf['jet_type']}_R{cf['subjet_radius']}"
    job_type = model.__class__.__name__
    name     = f"{job_type}_{cf['time']}_{suffix}"
    id       = f"{group}_{name}"
    root_dir = f"./result"
    if not os.path.isdir(root_dir):
        os.makedirs(root_dir)

    # wandb logger setup
    if cf["wandb"]:
        try:
            # check runs
            api = wandb.Api()
            for run in api.runs(f"ntuyianchen/{project}"):
                if f"{group}_{job_type}" in run.id and suffix in run.id:
                    run = api.run(f"ntuyianchen/{project}/"+run.id)
                    run.delete()
        except ValueError as e:
            print(e)
            print(f"Now creating new project {project}")
        wandb_logger = WandbLogger(project=project, group=group, job_type=job_type, name=name, id=id, save_dir=root_dir)
        wandb_logger.experiment.config.update(cf)
        wandb_logger.watch(model, log="all")

    # start lightning training
    logger   = wandb_logger if cf["wandb"] else None
    trainer  = L.Trainer(
        logger=logger, 
        accelerator       = cf["accelerator"], 
        max_epochs        = cf["max_epochs"], 
        fast_dev_run      = cf["fast_dev_run"],
        log_every_n_steps = cf["log_every_n_steps"],
        )
    litmodel = LitModel(model)
    
    if tune_batch:
        tuner = Tuner(trainer)
        tuner.scale_batch_size(litmodel, datamodule=data_module, mode="binsearch")
    trainer.fit(litmodel, datamodule=data_module)
    trainer.test(litmodel, datamodule=data_module)

    # finish wandb monitoring
    if cf["wandb"]:
        wandb.finish()

### Start training each model

In [10]:
# classical 2pcnn
data_pt_eta_phi = JetDataModule(events_func=events_pt_eta_phi, norm=False)
data_pt_eta_phi_norm = JetDataModule(events_func=events_pt_eta_phi, norm=True)
input_dim = data_pt_eta_phi.train_dataset[0].x.shape[1]
cf_2pcnn = {
    "gnn_in":input_dim, 
    "gnn_layers":cf["gnn_layers"],
    "gnn_aggr":"add", 
    "mlp_in":input_dim,
    "mlp_out":1, 
    "mlp_hidden":3*input_dim, 
    "mlp_layers":cf["mlp_layers"],
}
train(Classical2PCNNForward(**cf_2pcnn), data_module=data_pt_eta_phi, suffix="pt_eta_phi")
train(Classical2PCNNForward(**cf_2pcnn), data_module=data_pt_eta_phi_norm, suffix="pt_eta_phi_norm")

DataLog: ZprimeToZhToZinvhbb has 2980 events and QCD_HT2000toInf has 4917 events.
Choose num_data for each channel to be 1000 | Each channel  has num_train = 500, num_test = 500
DataLog: ZprimeToZhToZinvhbb has 2980 events and QCD_HT2000toInf has 4917 events.
Choose num_data for each channel to be 1000 | Each channel  has num_train = 500, num_test = 500
Could not find project t_4vec_2pcnn
Now creating new project t_4vec_2pcnn


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type                  | Params
--------------------------------------------------------
0 | model         | Classical2PCNNForward | 172   
1 | loss_function | BCEWithLogitsLoss     | 0     
--------------------------------------------------------
172       Trainable params
0         Non-trainable params
172       Total params
0.001     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name          | Type 

Epoch 49: 100%|██████████| 63/63 [00:00<00:00, 77.99it/s, v_num=_phi]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 63/63 [00:00<00:00, 77.54it/s, v_num=_phi]


...

Epoch 49: 100%|██████████| 63/63 [00:00<00:00, 80.04it/s, v_num=norm]


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 63/63 [00:00<00:00, 227.36it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.5890228152275085     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
test_acc_epoch,▁
test_acc_step,▅▅▄▄▅▄▇▅▅▆█▅▅▆▅▅▅▇▅▅▃▂▂▃▂▄▅▅▁▂▂▃▂▂▅▇▂▃▅▅
train_acc_epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▆▆▆▆▆▇▆▇▇▇▇▇▇▇▆▇▇██▇▆▇██
train_acc_step,▆▅▅▅▅▃▅▄▄▅▅▄▃▂▁▄▅▆▆▇█▄▆▇▄▇▆▆▄▇▅▅▄▆▆▅▅█▄▅
train_loss_epoch,█▆▆▅▅▅▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁
train_loss_step,▃▆▄▅▅▇▅▆▆▄▅▅▆▇▇▆▆▄▅▄▄▇▅▂▆▅▄▅▆▄▆▅▅▃▄▅▃▁▅█
trainer/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███▁▁▁▁▁
epoch,50
test_acc_epoch,0.58902
test_acc_step,0.375
